In [49]:
import csv
import pandas as pd
import json
####input your credentials here
consumer_key = 'HC3mhnbHYUNnS1Zfwnuv2C8dO'
consumer_secret = '9PEqSawfQSGKXytX5IJr4flOo0Bkp42h0zlXZc9jdzmDmR6fm3'
access_token = '1193698595658240000-7noD2IYnnoKPICU4LFtZLo7Ja9IUwz'
access_token_secret = 'oNeYKX2Bql8DWBOusAAPuM2N6qFFoXVyN6OQ4CrcegmRE'
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 

import numpy as np
import pandas as pd

In [50]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

In [51]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token,access_token_secret)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [52]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)



In [68]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])


        return df

In [69]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="realDonaldTrump", count=20)

    #print(dir(tweets[0]))
    #print(tweets[0].retweet_count)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    
    print(df.head(10))
    
    

                                              Tweets                   id  \
0  “The Chinese say (about Trump’s Trade War &amp...  1193897574899634176   
1  Shifty Adam Schiff will only release doctored ...  1193896275093184512   
2  The lawyer for the Whistleblower takes away al...  1193894144747429889   
3  So with one Rally by me at the end of the camp...  1193893631616270336   
4  Will be meeting with representatives of the Va...  1193891291953807360   
5                                HAPPY VETERANS DAY!  1193886728790171649   
6  RT @GOPChairwoman: While Democrats are focused...  1193881433338732547   
7    https://t.co/5NgNEZJm8G https://t.co/R5b47GeEp4  1193852154060197895   
8    https://t.co/LEjupyeKxw https://t.co/tox0vh5bbd  1193736163237994497   
9  RT @RepDougCollins: Rep. Swalwell claims, “for...  1193717656844742658   

   len                date              source  likes  retweets  
0  144 2019-11-11 14:25:38  Twitter for iPhone  22202      6337  
1  140 2019-11-11 14

In [70]:
df

,Tweets,id,len,date,source,likes,retweets
0,“The Chinese say (about Trump’s Trade War &amp...,1193897574899634176,144,2019-11-11 14:25:38,Twitter for iPhone,22202,6337
1,Shifty Adam Schiff will only release doctored ...,1193896275093184512,140,2019-11-11 14:20:28,Twitter for iPhone,37006,11306
2,The lawyer for the Whistleblower takes away al...,1193894144747429889,139,2019-11-11 14:12:00,Twitter for iPhone,32014,9237
3,So with one Rally by me at the end of the camp...,1193893631616270336,140,2019-11-11 14:09:57,Twitter for iPhone,27412,7016
4,Will be meeting with representatives of the Va...,1193891291953807360,140,2019-11-11 14:00:40,Twitter for iPhone,30638,7058
5,HAPPY VETERANS DAY!,1193886728790171649,19,2019-11-11 13:42:32,Twitter for iPhone,84533,17269
6,RT @GOPChairwoman: While Democrats are focused...,1193881433338732547,140,2019-11-11 13:21:29,Twitter for iPhone,0,5725
7,https://t.co/5NgNEZJm8G https://t.co/R5b47GeEp4,1193852154060197895,47,2019-11-11 11:25:08,Twitter for iPhone,35450,9808
8,https://t.co/LEjupyeKxw https://t.co/tox0vh5bbd,1193736163237994497,47,2019-11-11 03:44:14,Twitter for iPhone,33419,8804
9,"RT @RepDougCollins: Rep. Swalwell claims, “for...",1193717656844742658,140,2019-11-11 02:30:42,Twitter for iPhone,0,11021


In [72]:
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders"]
    fetched_tweets_filename = "tweets.json"

    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

{"created_at":"Mon Nov 11 20:21:11 +0000 2019","id":1193987054385999872,"id_str":"1193987054385999872","text":"RT @AnandWrites: I\u2019ve seen many interviews with billionaires about the candidacies of Elizabeth Warren and Bernie Sanders.\n\nHas anyone done\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1078680784721100800,"id_str":"1078680784721100800","name":"jeanette wright","screen_name":"GOPlastdays","location":null,"url":null,"description":"\ud83c\udff3\ufe0f\u200d\ud83c\udf08\ud83c\udf08\ud83c\uddf5\ud83c\uddf7\ud83d\udc99\ud83c\udf0a if you're gonna whine better buy some cheese","translator_type":"none","protected":false,"verified":false,"followers_count":4163,"friends_count":4839,"listed_count":1,"favourites_count":108568,"status

{"created_at":"Mon Nov 11 20:21:12 +0000 2019","id":1193987056478887936,"id_str":"1193987056478887936","text":"RT @gatewaypundit: Tulsi Gabbard's Lawyers Accuse Hillary Clinton of Defamation For 'Russian Asset' Claim - Demand 'Verbal' Retraction in P\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1177897885742047233,"id_str":"1177897885742047233","name":"Chris Richardson FR #Z","screen_name":"ChrisRi01983176","location":"France","url":null,"description":"100% french Patriot for frexit, against the EU mafia, for the Brexit, for American MAGA KEEP AMERICA FIRST Patriots and all veterans","translator_type":"none","protected":false,"verified":false,"followers_count":187,"friends_count":375,"listed_count":0,"favourites_count":596,"statuses_cou

{"created_at":"Mon Nov 11 20:21:12 +0000 2019","id":1193987058236178432,"id_str":"1193987058236178432","text":"@NateSilver538 Hillary Clinton needs to go away and stay away.  Bringing life to Tulsi\u2019s campaign, what an idiot.","display_text_range":[15,113],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":1193986420039454721,"in_reply_to_status_id_str":"1193986420039454721","in_reply_to_user_id":16017475,"in_reply_to_user_id_str":"16017475","in_reply_to_screen_name":"NateSilver538","user":{"id":293821136,"id_str":"293821136","name":"Linas","screen_name":"Mr_Linas","location":"Welcome to Da Bay","url":null,"description":"socialist who caucuses with the Democrats-investor-basketball fanatic-habitual hibernator-I'm not here to argue, just to tell you I'm right!","translator_type":"none","protected":false,"verified":false,"followers_count":104,"friends_count":638,"listed_c

{"created_at":"Mon Nov 11 20:21:13 +0000 2019","id":1193987060174118915,"id_str":"1193987060174118915","text":"RT @EaterSouls: Tulsi Gabbard's attorneys are demanding Hillary Clinton retract her assertion  that \u201csomebody who\u2019s currently in the Democr\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":822133872230105092,"id_str":"822133872230105092","name":"joe finnegan","screen_name":"joefinnegan91","location":"Villa Hills, KY","url":null,"description":"US ARMY 1960-1966 Now i am Old , I walk slower, but with as much purpose as when I walked in the RICE PADDIES...","translator_type":"none","protected":false,"verified":false,"followers_count":2250,"friends_count":2271,"listed_count":6,"favourites_count":21364,"statuses

{"created_at":"Mon Nov 11 20:21:13 +0000 2019","id":1193987062774427648,"id_str":"1193987062774427648","text":"RT @mcspocky: #CadetBoneSpurs loses in court.\nHey, @realDonaldtRump\u2026 You don't get to hide your state taxes!\n\ud83e\udd73\ud83c\udf7e\ud83c\udf89\ud83d\ude4c\ud83e\udd42\ud83c\udf87\ud83c\udf88\ud83c\udf7a\ud83c\udf7b\ud83c\udf77\ud83c\udf86\ud83c\udf78\ud83c\udf79\ud83e\udd73\ud83c\udf7e\ud83c\udf89\ud83d\ude4c\ud83e\udd42\ud83c\udf87\ud83c\udf88\ud83c\udf7a\ud83c\udf7b\ud83c\udf77\ud83c\udf86\ud83c\udf78\ud83c\udf79\ud83e\udd73\ud83c\udf7e\ud83c\udf89\ud83d\ude4c\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1145840742092292097,"id_str":"1145840742092292097","name":"@jfa","screen_name":"jfa54676101","location":null,"url

{"created_at":"Mon Nov 11 20:21:14 +0000 2019","id":1193987064561311747,"id_str":"1193987064561311747","text":"RT @RaheemKassam: Shit.\n\nSomeone put the UK on \u201csuicide watch\u201d. https:\/\/t.co\/MrxOgU1StT","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":958580808,"id_str":"958580808","name":"SOCIALIZED MEDICINE","screen_name":"MancunianMEDlC","location":null,"url":"http:\/\/SOCIALIZEDMEDICINE.WORDPRESS.COM","description":"Patient Safety Expert. I've seen an older patient given as little as 0.5mg MIDAZOLAM for day procedure who then had to be admitted to hospital overnight.","translator_type":"none","protected":false,"verified":false,"followers_count":6129,"friends_count":6575,"listed_count":5,"favourites_count":55659,"statuses_count":40022,"c

{"created_at":"Mon Nov 11 20:21:14 +0000 2019","id":1193987066113282048,"id_str":"1193987066113282048","text":"T\u2019yeah....NOPE. \ud83d\ude44","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":21126461,"id_str":"21126461","name":"Amy Alexander","screen_name":"AmyAlex63","location":"DC\/SF","url":"http:\/\/www.amyalexanderink.com","description":"Promoting Knowledge, Identity, Vision, Compassion, Justice, Reality and Humor. Opinions here are mine, RTs = Take a Look.","translator_type":"none","protected":false,"verified":false,"followers_count":2818,"friends_count":1964,"listed_count":271,"favourites_count":37750,"statuses_count":82235,"created_at":"Tue Feb 17 20:28:09 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":false,

{"created_at":"Mon Nov 11 20:21:15 +0000 2019","id":1193987071817310208,"id_str":"1193987071817310208","text":"RT @funder: Barack Obama: It's about how we treat our Veterans every single day of the year. It's about making sure they have the care they\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":269170900,"id_str":"269170900","name":"Meliannos","screen_name":"meliannos","location":"Nowhere you want to be, USA","url":null,"description":"\u2022 STOP \u2022 THINK \u2022 RESIST \u2022 Progressive Liberal Atheist \u2014 End the #WarOnWomen \u2014 Stop Radical Christianity \u2014 #SaveMedicare \u2014 We get the government we vote for!","translator_type":"none","protected":false,"verified":false,"followers_count":4352,"friends_coun

{"created_at":"Mon Nov 11 20:21:16 +0000 2019","id":1193987074610872320,"id_str":"1193987074610872320","text":"RT @JavierArocaA: Barack Obama jam\u00e1s hubiera pactado con el Ku Klux Klan,  ni siquiera con el Tea Party. #GraciasAlbert","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":479881590,"id_str":"479881590","name":"jreal\ud83d\udc94\u270a\ud83d\udc9c","screen_name":"JrealJavireal","location":"Barcelona, Espa\u00f1a","url":null,"description":"Esto no lo cambian ellos, lo haremos nosotros! Falta honestidad, coherencia y dignidad. Costar\u00e1,pero NO desfallezcas, \nPiensa y reflexiona antes de contestar.\ud83e\udd14","translator_type":"none","protected":false,"verified":false,"followers_count":4530,"friends_count":4992,"liste

KeyboardInterrupt: 

In [78]:
tweets = []
for line in open('tweets.json', 'r'):
    tweets.append(json.loads(line))

In [86]:
for i in [tweet['text'] for tweet in tweets]:
    print(i)
    print('\n\n\n')

RT @AnandWrites: I’ve seen many interviews with billionaires about the candidacies of Elizabeth Warren and Bernie Sanders.

Has anyone done…




RT @Wonderbitch82: First, it was Bernie Sanders. 

Second, it was Elizabeth Warren.

Now, it is Pete Buttigieg who feel the need to attack…




RT @patrickwintour: UK government's delay of Russian influence report is inexplicable, damaging and shaming, says Hillary Clinton.  https:/…




RT @gatewaypundit: Tulsi Gabbard's Lawyers Accuse Hillary Clinton of Defamation For 'Russian Asset' Claim - Demand 'Verbal' Retraction in P…




RT @PepperOceanna: "If I'm in the White House, she will play a very, very important role. No question. One way or another, absolutely," ~ @…




@NateSilver538 Hillary Clinton needs to go away and stay away.  Bringing life to Tulsi’s campaign, what an idiot.




RT @jonathanchait: Gabbard is falsely characterizing Clinton's comments -- she said Republicans, not Russians were grooming her for a 3rd p…




RT @EaterSoul